# NeuroFetal AI - Modular Training Pipeline

**Version 3.2** - Automated Secrets & TFLite Auto-Push

This notebook acts as a central launcher for the modular scripts in the codebase. It ensures consistency between local development and Cloud training.

### Steps:
1.  **Setup Environment**: Clone repo & install dependencies (Uses Colab Secrets).
2.  **Data Ingestion**: Process raw PhysioNet data.
3.  **Train**: Run the deep learning training pipeline.
4.  **Evaluate**: Run ensemble and uncertainty metrics.
5.  **Serve**: Launch the dashboard (optional).
6.  **Deploy**: Convert to TFLite and push to GitHub.

## 1. Setup Environment

In [1]:
from google.colab import userdata
import os

# 1. GitHub Authentication
GITHUB_REPO = "Krishna200608/NeuroFetal-AI"

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✓ GitHub Token loaded from Secrets.")
except Exception as e:
    print("⚠️ Error loading GITHUB_TOKEN from Secrets. Falling back to manual input.")
    from getpass import getpass
    GITHUB_TOKEN = getpass("Enter GitHub Personal Access Token (PAT): ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO

✓ GitHub Token loaded from Secrets.


In [2]:
# 2. Clone Repository
import shutil
import os

# CRITICAL FIX: Reset to /content before deleting the repo folder
# This prevents 'shell-init: error retrieving current directory'
try:
    os.chdir("/content")
except:
    pass

# Clean up any previous clone to avoid conflicts
if os.path.exists("/content/NeuroFetal-AI"):
    shutil.rmtree("/content/NeuroFetal-AI")

print("Cloning repository...")
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git

# Set paths
os.chdir("/content/NeuroFetal-AI")
print("Cloned successfully!")

Cloning repository...
Cloning into 'NeuroFetal-AI'...
remote: Enumerating objects: 2046, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 2046 (delta 41), reused 31 (delta 17), pack-reused 1970 (from 2)
Receiving objects: 100% (2046/2046), 524.99 MiB | 23.48 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.
Updating files: 100% (1187/1187), done.
Cloned successfully!


In [3]:
# 3. Install Dependencies
print("Installing libraries...")
!pip install -q wfdb shap scipy imbalanced-learn pyngrok filterpy scikit-learn matplotlib seaborn pandas numpy tensorflow streamlit plotly python-dotenv
print("Dependencies installed.")

Installing libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
Dependencies installed.


## 2. Data Ingestion
Processes raw `.dat`/`.hea` files into clean `.npy` arrays for training.

In [4]:
# Run the data ingestion script
!python Code/scripts/data_ingestion.py

Found 552 records.
Processed 100 records...
Processed 200 records...
Processed 300 records...
Processed 400 records...
Processed 500 records...
Processing complete. Processed 552 patients into 2760 slices.
Shapes: X_fhr (2760, 1200), X_uc (2760, 1200), X_tabular (2760, 3), y (2760,)
Class balance: 200 compromised / 2760 total


## 2.5. Self-Supervised Pretraining (New)
Train the Masked Autoencoder (MAE) on the FHR data to learn robust features before supervision.
This saves encoder weights to `Code/models/pretrained_fhr_encoder.weights.h5`.

In [5]:
!git pull origin main

From https://github.com/Krishna200608/NeuroFetal-AI
 * branch            main       -> FETCH_HEAD
Already up to date.


In [6]:
# Run the SSL pretraining script
!python Code/scripts/pretrain.py

2026-02-08 06:24:24.372173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770531864.391849     765 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770531864.397711     765 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770531864.412866     765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770531864.412900     765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770531864.412904     765 computation_placer.cc:177] computation placer alr

## 3. Training
Train the Tri-Modal Attention Fusion ResNet using 5-Fold Cross-Validation.
This script automatically handles:
*   Class Balancing (SMOTE)
*   Feature Extraction (CSP)
*   Model Checkpointing (saving best `.keras` files)

In [7]:
# Training Loop with Git Push
import os

folds = 5
epochs = 150
batch_size = 64

for fold in range(1, folds + 1):
    print(f"\n{'='*40}")
    print(f"Training Fold {fold}/{folds}")
    print(f"{'='*40}")
    
    # Run training for this fold
    !python Code/scripts/train.py --fold {fold} --epochs {epochs} --batch_size {batch_size}
    
    # Push to GitHub
    model_path = f"Code/models/enhanced_model_fold_{fold}.keras"
    if os.path.exists(model_path):
        print(f"Pushing model for Fold {fold} to GitHub...")
        !git add {model_path}
        !git commit -m "Auto-save: Trained model for Fold {fold}"
        !git push origin main
        print(f"✓ Model for Fold {fold} pushed successfully.")
    else:
        print(f"⚠️ Model file not found: {model_path}")

# Evaluation after training
print("\nRunning Ensemble Evaluation (Rank Averaging)...")
!python Code/scripts/evaluate_ensemble.py

print("\nRunning Uncertainty Quantification...")
!python Code/scripts/evaluate_uncertainty.py

2026-02-08 06:27:10.161677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770532030.202333    1950 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770532030.212286    1950 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770532030.232550    1950 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770532030.232582    1950 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770532030.232590    1950 computation_placer.cc:177] computation placer alr

In [8]:
!git add .

In [12]:
!git config --global user.email "krishnasikheriya001@gmail.com"
!git config --global user.name "Krishna200608"

In [13]:
!git commit -m "Updates from the colab"

[main fd694c1] Updates from the colab
 7 files changed, 49 insertions(+)
 create mode 100644 Code/models/pretrained_fhr_encoder.weights.h5
 create mode 100644 Reports/training_logs/training_log_20260208_071829.json


In [14]:
!git push origin main

Enumerating objects: 23, done.
Counting objects: 100% (23/23), done.
Delta compression using up to 2 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 172.66 MiB | 9.66 MiB/s, done.
Total 13 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/Krishna200608/NeuroFetal-AI.git
   e674aef..fd694c1  main -> main


In [11]:
from IPython.display import Javascript, display

display(Javascript("""
var audio = new Audio("https://actions.google.com/sounds/v1/alarms/alarm_clock.ogg");
audio.play();
"""))


<IPython.core.display.Javascript object>

## 4. Advanced Evaluation
Generate metrics for:
1.  **Ensemble Performance**: Rank Averaging across folds (AUC maximization).
2.  **Uncertainty Quantification**: Monte Carlo Dropout confidence scores.

In [16]:
!git pull origin main

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 693 bytes | 346.00 KiB/s, done.
From https://github.com/Krishna200608/NeuroFetal-AI
 * branch            main       -> FETCH_HEAD
   fd694c1..a8fb69f  main       -> origin/main
Updating fd694c1..a8fb69f
Fast-forward
 Code/utils/attention_blocks.py | 1 +
 1 file changed, 1 insertion(+)


In [17]:
print("Running Ensemble Evaluation (Rank Averaging)...")
!python Code/scripts/evaluate_ensemble.py

print("\nRunning Uncertainty Quantification...")
!python Code/scripts/evaluate_uncertainty.py

Running Ensemble Evaluation (Rank Averaging)...
2026-02-08 07:35:34.115963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770536134.136577   24290 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770536134.142501   24290 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770536134.166478   24290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770536134.166510   24290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770536134.166515   24290 c

## 5. Launch Dashboard (Optional)
Run the Streamlit app directly from Colab using `ngrok`.
**Note**: You need an `NGROK_AUTH_TOKEN` set in your Colab Secrets.

In [19]:
!pip install streamlit plotly python-dotenv pyngrok

In [ ]:
from google.colab import userdata

try:
    auth_token = userdata.get('NGROK_AUTH_TOKEN')
    print("✓ Ngrok Token loaded from Secrets.")
except Exception as e:
    print("⚠️ Error loading NGROK_AUTH_TOKEN from Secrets. Falling back to manual input.")
    from getpass import getpass
    auth_token = getpass("Enter Ngrok Auth Token manually: ")

if auth_token:
    with open("Code/.env", "w") as f:
        f.write(f"NGROK_AUTH_TOKEN={auth_token}\n")

print("Launching Streamlit App...")
!python Code/run_app.py

⚠️ Error loading NGROK_AUTH_TOKEN from Secrets. Falling back to manual input.
Enter Ngrok Auth Token manually: ··········
Launching Streamlit App...
Authenticating with ngrok...
Starting Streamlit Server...
Attempting to open public tunnel...

   DASHBOARD LIVE AT: https://beauteously-uncaped-dario.ngrok-free.dev
   LOCAL ADDRESS:     http://localhost:8501

Press Ctrl+C to stop the server.


## 6. Convert to TFLite (Mobile Deployment)
Convert the best trained model to TFLite format and **PUSH to GitHub** automatically.
You don't need to download anything manually.

In [18]:
print("Converting model to TFLite...")
!python Code/scripts/convert_to_tflite.py

print("\nPushing TFLite models to GitHub...")
# Configure Git identity (required for commit)
!git config --global user.email "krishnasikheriya001@gmail.com"
!git config --global user.name "Krishna200608"

# Add TFLite models
!git add Code/models/tflite/*.tflite

# Commit and Push
# We use '|| true' to prevent error if nothing to commit (e.g. running twice)
!git commit -m "chore: Auto-update TFLite models from Colab" || true
!git push origin main

print("✓ Models pushed to GitHub successfully! Check the repo.")

Converting model to TFLite...
2026-02-08 07:40:57.489768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770536457.508043   25873 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770536457.513524   25873 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770536457.527828   25873 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770536457.527857   25873 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770536457.527861   25873 computation_placer.